In [6]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used
sys.path.append("./prediction_models/input/prostate-cancer-grade-assessment/")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import *
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
# print(device)

## customized package
from input.inputPipeline import *
from model.resnext_ssl import Model_Infer as Model

In [2]:
data_dir = './input/panda-32x256x256-tiles-data/train'
csv_file = './input/panda-16x128x128-tiles-data/4_fold_train.csv'
# SAMPLE = './input/prostate-cancer-grade-assessment/sample_submission.csv'

In [3]:
mean = torch.tensor([0.90949707, 0.8188697, 0.87795304])
std = torch.tensor([0.36357649, 0.49984502, 0.40477625])

In [7]:
models = []
weights = [f'./train/weights/Resnext50_reg_medreso_12patch/Resnext50_reg_medreso_12patch_{i}_best.pth.tar' for i in range(4)]
for path in weights:
#     state_dict = torch.load(path,map_location=torch.device('cpu'))
    state_dict = torch.load(path)
    model = Model(n = 1)
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.to(device)
    models.append(model)

del state_dict

In [8]:
sz = 256
bs = 1
folds = 4
tsfm = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize(mean=mean,
                              std=std)])
dataset = PandaPatchDataset(csv_file, data_dir, transform=tsfm)
crossValData = crossValDataloader(csv_file, dataset, bs)

In [ ]:
val_label, val_preds = [], []
for fold in range(folds):
    _, valloader = crossValData(fold)
    model = models[fold]
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(valloader, desc='valIter'), start=0):
#             if i > 5:
#                 break
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            outputs = model(inputs.cuda())
            outputs = outputs.squeeze(dim=1)  # for regression
            val_label.append(labels.cpu())
            val_preds.append(outputs.round().to(torch.int64).clamp_(min=0, max=5).cpu())

In [ ]:
val_preds = torch.cat(val_preds, 0) # for regression
val_label = torch.cat(val_label)
kappa = cohen_kappa_score(val_label.view(-1), val_preds.view(-1), weights='quadratic')
cm = confusion_matrix(val_label.view(-1), val_preds.view(-1))
ncm = cm / cm.astype(np.float).sum(axis=1)
print("The average kappa score is {}.".format(kappa))

In [ ]:
df_cm = pd.DataFrame(ncm, range(6), range(6))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.show()